<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/2.1-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

El objetivo de este notebook es enriquecer el dataset user_reviews con una nueva columna llamada 'sentiment_analysis'. Esta columna contendrá el resultado de aplicar un análisis de sentimiento con NLP a las reseñas de los juegos escritas por los usuarios. De esta manera, podremos explorar la opinión de los usuarios sobre los diferentes juegos. Además, vamos a hacer un merge entre los distintos datasets para obtener una base de datos única y más eficiente para cada consulta, que nos permitirá optimizar el rendimiento de la API y reducir el tamaño de nuestra base de datos.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [1]:
import sys
import os
import pandas as pd
import textblob
from textblob import TextBlob

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"TextBlob version: {textblob.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3
TextBlob version: 0.17.1


## 1 Extracción

En esta sección, extraemos los datos de los archivos `steam_games`, `user_items` y `user_reviews` que estan en formato parquet.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [2]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'data/interim/{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['steam_games','user_items', 'user_reviews']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_steam_games = dataframes['steam_games']
df_user_items = dataframes['user_items']
df_user_reviews = dataframes['user_reviews']

## 2 Análisis de sentimiento

El análisis de sentimiento consiste en asignar una etiqueta numérica a cada reseña, según el tono o la actitud que expresa el texto. Usaremos la siguiente escala:

* 0: si la reseña es **negativa**, es decir, si el usuario muestra insatisfacción, disgusto o decepción con el juego.
* 1: si la reseña es **neutral**, es decir, si el usuario muestra indiferencia, objetividad o ausencia de emoción con el juego.
* 2: si la reseña es **positiva**, es decir, si el usuario muestra satisfacción, gusto o admiración con el juego.



### 2.1 Función `sentiment_analysis`

Para realizar el análisis de sentimiento con NLP a las reseñas de los juegos, crearemos una función usando la librería TextBlob que se considera facil de usar y muy intuitiva. Usaremos la polaridad que es una medida numérica que indica si el texto es negativo o positivo, según el tono o la actitud que expresa. La polaridad varía entre -1 y 1, donde -1 significa muy negativo, 0 significa neutro y 1 significa muy positivo.

In [3]:
def sentiment_analysis(review):
    # Si la reseña está ausente, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Calcula la polaridad de la reseña usando TextBlob
    polarity = TextBlob(review).sentiment.polarity

    # Retorna 0 (malo) si la polaridad es menor que 0, 2 (positivo) si la polaridad es mayor que 0, y 1 (neutral) en caso contrario
    if polarity < 0:
        return 0
    elif polarity > 0:
        return 2
    else:
        return 1

* Aplicamos la función a la columna `review`.

In [4]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(sentiment_analysis)

* Veamos algunos ejemplos:

In [6]:
df_user_reviews[['review','sentiment_analysis']].sample(5)

,review,sentiment_analysis
4594,FEKIN GET THE GAME!!! (If you have a good grap...,0
2993,gud,1
20997,Just a short review.Survarium is a post apocol...,1
14682,"You only know his name, not his story!",1
10938,"WARNING: DO NOT SUBSCRIBE!!Hi guys, I'm diabz....",2


### 2.2 Eliminación de la columna `review`

La nueva columna `sentiment_analysis` reemplazará a la columna `review` en el dataset `user_reviews`, para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [7]:
df_user_reviews.drop('review', axis=1, inplace=True)
df_user_reviews.head()

,item_id,recommend,user_id,posted_year,sentiment_analysis
0,1250,True,76561197970982479,2011,2
1,22200,True,76561197970982479,2011,2
2,43110,True,76561197970982479,2011,2
3,251610,True,js41637,2014,2
4,227300,True,js41637,2013,0


## 3 Diseño y estructura de las bases de datos para los endpoints de la API

En esta sección, nuestro objetivo es crear diferentes dataset a modo de pseudo base de datos para las funciones que se usarán en los endpoints de la API. De esta manera, podremos acceder a los datos que necesitamos de forma rápida y eficiente, sin tener que cargar toda la información para así, optimizar el rendimiento de la API.

### 3.1 Endpoints 1 y 2

Estos endpoints comparten información en común, por lo que podemos crear un solo dataset para ambos.

#### 3.1.1 Endpoint 1

def **PlayTimeGenre( *`genero` : str* )**:
    Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

``` js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```



#### 3.1.2 Endpoint 2

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:
```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```

#### 3.1.4 Pseudo Database 1

Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_items` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`genres`,`release_year` del DataFrame `steam_games`. También `item_id`, `user_id` y `playtime_forever` del DataFrame `user_items`.

* Primero, seleccionamos solo las columnas necesarias:

In [9]:
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [10]:
# Nos aseguramos que el tipo de dato en 'genres' sea string.
df_steam_games['genres'] = df_steam_games['genres'].astype(str)

df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre `steam_games` y `user_items` en la columna `item_id`.

In [24]:
df_pseudo_db1 = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_pseudo_db1.head()

,item_id,genres,release_year,user_id,playtime_forever
0,282010,['Racing' 'Action' 'Indie'],1997,UTNerd24,0.083333
1,282010,['Racing' 'Action' 'Indie'],1997,I_DID_911_JUST_SAYING,0.000000
2,282010,['Racing' 'Action' 'Indie'],1997,76561197962104795,0.000000
3,282010,['Racing' 'Action' 'Indie'],1997,r3ap3r78,0.000000
4,282010,['Racing' 'Action' 'Indie'],1997,saint556,0.216667


In [25]:
df_pseudo_db1.shape

(4244831, 5)

- Con el fin de ahorrar recursos, solo usaremos los registros de juegos que cumplan con las siguientes condiciones: tener un `release_year` y `genres` válidos, y haber sido jugados al menos una vez.

In [26]:
df_pseudo_db1 = df_pseudo_db1[(df_pseudo_db1['release_year'] != 'unknown') & (df_pseudo_db1['playtime_forever'] > 0) & ~df_pseudo_db1['genres'].str.contains('unknown')].reset_index(drop=True)

* Agrupamos para obtener la suma de `playtime_forever` por `genres`	, `user_id`	y `release_year`.

In [27]:
df_pseudo_db1 = df_pseudo_db1.groupby(['user_id', 'release_year', 'genres'])['playtime_forever'].sum().reset_index()
df_pseudo_db1.head()

,user_id,release_year,genres,playtime_forever
0,--000--,2006,['Simulation' 'Action' 'Indie' 'Adventure' 'Ma...,15.416667
1,--000--,2009,['Action' 'Adventure'],88.816667
2,--000--,2010,['Free to Play' 'Indie' 'Action'],0.366667
3,--000--,2011,['Action' 'Adventure' 'RPG'],32.033333
4,--000--,2011,['Action' 'Strategy' 'Simulation' 'RPG' 'Indie...,11.083333


- Por último, para optimizar el uso de la memoria en el deploy, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [28]:
df_pseudo_db1['release_year'] = df_pseudo_db1['release_year'].astype('int16')
df_pseudo_db1['playtime_forever'] = df_pseudo_db1['playtime_forever'].astype('float32')
df_pseudo_db1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2593865 entries, 0 to 2593864
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   release_year      int16  
 2   genres            object 
 3   playtime_forever  float32
dtypes: float32(1), int16(1), object(2)
memory usage: 54.4+ MB


### 3.2 Endpoints 3, 4 y 5.

#### 3.2.1 Endpoint 3

+ def **UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  

Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```


#### 3.2.2 Endpoint 4

+ def **UsersWorstDeveloper( *`año` : int* )**:
   Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```

#### 3.2.3 Endpoint 5

def **sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:
```js
{
   "Valve":[
      Negative = 182,
      Neutral = 120,
      Positive = 278
   ]
}
```


#### 3.1.4 Pseudo Database 2

- Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_reviews` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`item_name`,`developer` del DataFrame `steam_games`. También `item_id`, `recommend`, `sentiment_analysis` y `posted_year` del DataFrame `user_reviews`.

- Primero, seleccionamos las columnas necesarias:

In [29]:
steam_games_columns = ['item_id', 'item_name', 'developer']
user_reviews_columns = ['item_id', 'recommend','sentiment_analysis','posted_year']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [30]:
df_games_subset = df_steam_games[steam_games_columns]
df_reviews_subset = df_user_reviews[user_reviews_columns]

* Luego, hacemos un merge entre los subsets `steam_games` y `user_reviews` en la columna `item_id`.

In [35]:
df_pseudo_db2 = pd.merge(df_games_subset, df_reviews_subset, on='item_id')
df_pseudo_db2.head()

,item_id,item_name,developer,recommend,sentiment_analysis,posted_year
0,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
1,70,Half-Life,Valve,True,0,2015
2,70,Half-Life,Valve,True,0,2011
3,70,Half-Life,Valve,True,0,2014
4,70,Half-Life,Valve,True,2,2013


- Para ahorrar recursos, solo utilizaremos registros de juegos que tengan un review con `posted_year` conocido y eliminamos la columna `item_id` que ya no es necesaria.

In [36]:
df_pseudo_db2 = df_pseudo_db2[df_pseudo_db2['posted_year'] != 'unknown'].reset_index(drop=True)
df_pseudo_db2.drop('item_id',axis=1, inplace=True)
df_pseudo_db2.head()

,item_name,developer,recommend,sentiment_analysis,posted_year
0,Half-Life,Valve,True,0,2015
1,Half-Life,Valve,True,0,2011
2,Half-Life,Valve,True,0,2014
3,Half-Life,Valve,True,2,2013
4,Half-Life,Valve,True,0,2013


- Por último, para optimizar el uso de la memoria en el deploy de la API, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [37]:
df_pseudo_db2['sentiment_analysis'] = df_pseudo_db2['sentiment_analysis'].astype('int8')
df_pseudo_db2['posted_year'] = df_pseudo_db2['posted_year'].astype('int16')
df_pseudo_db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44112 entries, 0 to 44111
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_name           44112 non-null  object
 1   developer           44112 non-null  object
 2   recommend           44112 non-null  bool  
 3   sentiment_analysis  44112 non-null  int8  
 4   posted_year         44112 non-null  int16 
dtypes: bool(1), int16(1), int8(1), object(2)
memory usage: 861.7+ KB


## 4 Carga

Finalmente, en esta sección cargamos nuestros datos transformados para los endpoints que se consumirán en la API a su destino final. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [39]:
# Nombres correspondientes a cada DataFrame
dfs = [df_pseudo_db1, df_pseudo_db2, df_user_reviews]
names = ['pseudo-db1.parquet', 'pseudo-db2.parquet', 'user_sentiment_analysis.parquet' ]

for dfs, n in zip(dfs, names):
    # Definimos la ruta del directorio
    folder_path = f'data/processed/'

    # Verificamos si el folder_path existe
    if not os.path.exists(folder_path):
        # Si no existe, lo creamos
        os.makedirs(folder_path)

    # Definimos la ruta completa del archivo
    path = os.path.join(folder_path, n)

    # Guardamos el DataFrame como un archivo parquet
    dfs.to_parquet(path, engine='pyarrow', compression='snappy')

    print(f"'{n}' fue guardado correctamente en '{folder_path}'")

'pseudo-db1.parquet' fue guardado correctamente en 'data/processed/'
'pseudo-db2.parquet' fue guardado correctamente en 'data/processed/'
'user_sentiment_analysis.parquet' fue guardado correctamente en 'data/processed/'
